In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random , logstash , logging , pymongo



In [2]:


host = '192.168.114.130'
client = "mongodb://" + host + ":27017/"
# 多執行續下 可能被 dead lock 
myclient = pymongo.MongoClient(client, connect=False)
mydb = myclient["mydatabase"]
mycol = mydb["Mobile01"]


In [ ]:
# pipeline 
logger = logging.getLogger('python-logstash-logger')
logger.setLevel(logging.INFO)
host_demo = "logstash"
logger.addHandler(logstash.TCPLogstashHandler(host_demo,5959))




In [3]:
proxylist = [{"http": "37.187.120.123:80"},
             {"https": "206.189.36.198:8080"},
             {"https": "178.128.31.153:8080"},
             {"http": "167.114.180.102:8080"},
             {"http": "104.131.214.218:80"},
             {"http": "167.114.196.153:80"}]
header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}



def get_connect(url):
    i = 0
    
    while True :
        
        try :
            resp = requests.get(url,headers=header,proxies=proxylist[i])          
            resp.encoding = "utf-8"
            time.sleep(random.randint(1,4))
           
            
        except requests.exceptions.ConnectionError :             
            #logger.warning("Proxy Error",extra={"proxy":proxylist[i]})
            
            i = (i + 1) % 6
            continue

            
            
        if resp : 
            if resp.status_code == 200 :
                code = "HTTP response code = " + str(resp.status_code)
                #logger.info(code,extra={"url" : url, "time_use" : resp.elapsed.total_seconds()})
                #print(proxylist[i])
                break         
        
        
    return resp.text 
    

In [4]:
url = "https://www.mobile01.com/waypointlist.php?list=1&c=0&s=desc&pid=2&p=1"

soup = BeautifulSoup(get_connect(url))

In [8]:
table = soup.find_all("tr")
table[0]

<tr>
<td class="icon" width="200"><a href="waypointdetail.php?id=32535"><img border="0" height="150" src="//attach.mobile01.com/waypoint/mobile01-dba7b25a2157b4155d29721a91370806.jpg" width="200"/></a></td>
<td class="detail"><p class="title"><a class="rb16" href="waypointdetail.php?id=32535">臺北市 北投區 溫泉博物館 — 鬱鬱蔥蔥中的紅磚建築，英式建築中的日式溫泉鄉</a></p>
<p class="desc">北投溫泉博物館，在日治時期為「北投公共浴場」，完工於1913年6月，距今已有105年。 建築本體為雙層和洋折衷風格建築，外觀仿英式建築，而內部則部分為和式空間，最初的用途就是溫泉浴室。 日本人愛泡溫泉舉世聞名，在發現北投的優質磺泉後，即儘...</p>
<p class="info">景點分類: <a href="waypointlist.php?pid=2">台北市</a> - 風景 - 博物館或紀念館<br/>
                    發佈日期: 2019-01-11 <a class="bk12u" href="userinfo.php?id=537205">老狗5550</a><br/>
                    無線網路: 有<br/>
</p></td>
</tr>

In [24]:
# 取得列表內容
for t in table:
    title = t.find("p", class_="title").text
    href = "https://www.mobile01.com/" + t.find("p", class_="title").find("a")["href"]
    sp = t.find("p", class_="info").text.split("-")[1]
    

title :  臺北市 北投區 溫泉博物館 — 鬱鬱蔥蔥中的紅磚建築，英式建築中的日式溫泉鄉
href :  https://www.mobile01.com/waypointdetail.php?id=32535
sp :   風景 
title :  【食記】台北捷運公館站。初牛炭火直燒丼飯(公館店)~免費續飯 湯無限喝 免服務費
href :  https://www.mobile01.com/waypointdetail.php?id=32526
sp :   餐飲 
title :  臺北市 士林區 2018士林官邸公園「菊世遨遊」菊展 — 萬紫千紅之花團錦簇，快賞菊花去 ~~
href :  https://www.mobile01.com/waypointdetail.php?id=32523
sp :   風景 
title :  台北市士林區[小南鄭記台南碗粿(士林店)]士林夜市/士林捷運站 得獎碗粿 ~ 台南風味~ 古早味~口味佳~
href :  https://www.mobile01.com/waypointdetail.php?id=32522
sp :   餐飲 
title :  [台北食紀] 內湖「加賀伊壽司割烹」日式無菜單料理 初訪
href :  https://www.mobile01.com/waypointdetail.php?id=32490
sp :   餐飲 
title :  台北市內湖區 [維堤咖啡] 推薦榛果巧克力超好喝! 店內裝潢氣氛佳~ 適合下午聊天享受時光
href :  https://www.mobile01.com/waypointdetail.php?id=32487
sp :   餐飲 
title :  【台北市 中山區/捷運中山站】根 職人料理-充滿創意和驚喜的日本料理
href :  https://www.mobile01.com/waypointdetail.php?id=32476
sp :   餐飲 
title :  【台北市 內湖區/捷運西湖站】帝一頂級燒烤-帝王蟹.和牛.伊比利豬通通吃到飽
href :  https://www.mobile01.com/waypointdetail.php?id=32470
sp :   餐飲 
title :  擺盤與色澤都讓人大快朵頤

In [26]:
href = "https://www.mobile01.com/waypointdetail.php?id=32415"
soup = BeautifulSoup(get_connect(href))

In [27]:
soup

<!DOCTYPE html>

<html lang="zh-TW">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="【台北市 內湖區/近內湖Costco】燒肉的名門 赤虎-大口品嚐日式燒肉超爽快" name="title">
<meta content="Mobile01 景點 - 文章出處Ally 愛玩樂http://ally701.pixnet.net/blog/post/466537493-%E3%80%90%E5%8F%B0%E5%8C%97%E5%B8%82-%E5%85%A7%E6%B9%96%E5%8D%80-%E8%BF%91%E5%85%A7%E6%B9%96costco..." name="description"/>
<meta content="台北市, 景點, 美食, 遊記, 餐廳, 民宿, 旅遊, 座標, 地圖, 小吃, 【台北市 內湖區/近內湖Costco】燒肉的名門 赤虎-大口品嚐日式燒肉超爽快" name="news_keywords"/>
<meta content="index,follow" name="robots"/>
<meta content="【台北市 內湖區/近內湖Costco】燒肉的名門 赤虎-大口品嚐日式燒肉超爽快" property="og:title"/>
<meta content="https://www.mobile01.com/waypointdetail.php?id=32415" property="og:url"/>
<meta content="article" property="og:type"/>
<meta content="Mobile01 景點 - 文章出處Ally 愛玩樂http://ally701.pixnet.net/blog/post/466537493-%E3%80%90%E5%8F%B0%E5%8C%97%E5%B8%82-%E5%85%A7%E6%B9%96%E5%8D%80-%E8%BF%91%E5%85%A7%E6%B9%96costco..." property="og:d